# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init('/home/kajili/spark-2.4.5-bin-hadoop2.7')
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('k-means-clustering-consult').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

# Loads data.
data = spark.read.csv("hack_data.csv",header=True,inferSchema=True)

In [3]:
data.show(5, False)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location              |WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+
|8.0                    |391.09           |1              |2.96             |7.0            |Slovenia              |72.37           |
|20.0                   |720.99           |0              |3.04             |9.0            |British Virgin Islands|69.08           |
|31.0                   |356.32           |1              |3.71             |8.0            |Tokelau               |70.58           |
|2.0                    |228.08           |1              |2.48             |8.0            |Bolivia               |70.8            |
|20.0                   |408.5            |0              |3.5

In [4]:
data.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [5]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [8]:
feature_cols = ['Session_Connection_Time',
                'Bytes Transferred',
                'Kali_Trace_Used',
                'Servers_Corrupted',
                'Pages_Corrupted',
                'WPM_Typing_Speed']


vec_assembler = VectorAssembler(inputCols = feature_cols, outputCol='features')

In [9]:
final_data = vec_assembler.transform(data)

In [10]:
final_data.show(5,False)

+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+--------------------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|Location              |WPM_Typing_Speed|features                        |
+-----------------------+-----------------+---------------+-----------------+---------------+----------------------+----------------+--------------------------------+
|8.0                    |391.09           |1              |2.96             |7.0            |Slovenia              |72.37           |[8.0,391.09,1.0,2.96,7.0,72.37] |
|20.0                   |720.99           |0              |3.04             |9.0            |British Virgin Islands|69.08           |[20.0,720.99,0.0,3.04,9.0,69.08]|
|31.0                   |356.32           |1              |3.71             |8.0            |Tokelau               |70.58           |[31.0,356.32,1.0,3.71,8.0,70.58]

In [11]:
# Features column is the only really important one. can keep rest of cols in final_data DataFrame though.
final_data.select('features').show(5,False)

+--------------------------------+
|features                        |
+--------------------------------+
|[8.0,391.09,1.0,2.96,7.0,72.37] |
|[20.0,720.99,0.0,3.04,9.0,69.08]|
|[31.0,356.32,1.0,3.71,8.0,70.58]|
|[2.0,228.08,1.0,2.48,8.0,70.8]  |
|[20.0,408.5,0.0,3.57,8.0,71.28] |
+--------------------------------+
only showing top 5 rows



In [12]:
# Trains a k-means model with k=2.
kmeans_obj_2 = KMeans(featuresCol='features',k=2)
hacker_model_2 = kmeans_obj_2.fit(final_data)

# Trains a k-means model with k=3.
kmeans_obj_3 = KMeans(featuresCol='features',k=3)
hacker_model_3 = kmeans_obj_3.fit(final_data)

In [13]:
# Evaluate clustering by computing Within Set Sum of Squared Errors for k=2.
wssse2 = hacker_model_2.computeCost(final_data)
print("Within Set Sum of Squared Errors = " + str(wssse2))

# Evaluate clustering by computing Within Set Sum of Squared Errors for k=3.
wssse3 = hacker_model_3.computeCost(final_data)
print("Within Set Sum of Squared Errors = " + str(wssse3))

Within Set Sum of Squared Errors = 6915193.171893887
Within Set Sum of Squared Errors = 4081151.044882576


In [14]:
# Shows the result for k=2.
centers2 = hacker_model_2.clusterCenters()
print("Cluster Centers for k=2: ")
for center in centers2:
    print(center)

print('\n')

# Shows the result for k=3.
centers3 = hacker_model_3.clusterCenters()
print("Cluster Centers for k=3: ")
for center in centers3:
    print(center)

Cluster Centers for k=2: 
[ 20.36470588 364.22370588   0.48823529   3.64952941   8.54705882
  67.45364706]
[4.00060976e+01 8.59157866e+02 5.36585366e-01 6.92634146e+00
 1.32134146e+01 4.68612195e+01]


Cluster Centers for k=3: 
[ 18.38461538 254.87307692   0.43956044   3.09978022   7.9010989
  70.51747253]
[4.09722222e+01 8.87257847e+02 5.20833333e-01 7.13319444e+00
 1.35347222e+01 4.55445833e+01]
[ 24.74747475 523.85191919   0.56565657   4.5159596    9.61616162
  62.39242424]


In [15]:
result2 = hacker_model_2.transform(final_data)

result3 = hacker_model_3.transform(final_data)

In [16]:
result2.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|         0|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|         1|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|[31.0,356.32,1.0,...|         0|
|               

In [17]:
result3.show(5)
# we only care about the predictions. so we will select those.

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|         2|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|         1|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|[31.0,356.32,1.0,...|         0|
|               

In [18]:
predictions2 = result2.select('prediction')

predictions3 = result3.select('prediction')

In [19]:
predictions2.show()

+----------+
|prediction|
+----------+
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [20]:
predictions3.show()

+----------+
|prediction|
+----------+
|         2|
|         1|
|         0|
|         0|
|         2|
|         2|
|         0|
|         0|
|         0|
|         2|
|         0|
|         0|
|         2|
|         0|
|         1|
|         0|
|         0|
|         0|
|         0|
|         2|
+----------+
only showing top 20 rows



In [21]:
predictions2.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  164|
|         0|  170|
+----------+-----+



In [22]:
predictions3.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  144|
|         2|   99|
|         0|   91|
+----------+-----+



# Answer:
- Because of this section of the problem statement we know that there must be an even number of attacks split among either 2 or 3 hackers:

```
One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.
```

- Our results clearly show that when k=2 for number of clusters the algorithm tries to make, that the count of hacks was about even for 2 hackers, and the result is not evenly split when trying to cluster with k=3.

# So, there are 2 hackers.